In [185]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import panel as pn
pn.extension()


In [186]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [187]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [188]:
# Reset the index to make 'coin_id' a column for plotting
df_plot = df_market_data.reset_index()

# Plot with a proper x-axis
df_plot.hvplot.line(
    x="coin_id",  # Use 'coin_id' as x-axis
    y=["price_change_percentage_24h", "price_change_percentage_7d"],  # Select numerical columns
    width=800,
    height=400,
    rot=90,
    xlabel="Cryptocurrency",
    ylabel="Percentage Change",
    title="Crypto Price Changes Over Time"
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

### Prepare the Data

In [189]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
# Select relevant numerical columns for scaling
numerical_columns = ["price_change_percentage_24h", "price_change_percentage_7d"]
df_market_scaled = df_market_data[numerical_columns]

# Initialize StandardScaler and apply transformation
scaler = StandardScaler()
df_market_scaled = scaler.fit_transform(df_market_scaled)

In [190]:
# Create a DataFrame with the scaled data
df_market_scaled = pd.DataFrame(df_market_scaled, columns=numerical_columns)

# Copy the crypto names from the original DataFrame
df_market_scaled["coin_id"] = df_market_data.index

# Set the coin_id column as index
df_market_scaled = df_market_scaled.set_index("coin_id")

# Display the scaled DataFrame
df_market_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d
coin_id,,
bitcoin,0.508529,0.493193
ethereum,0.185446,0.934445
tether,0.021774,-0.706337
ripple,-0.040764,-0.810928
bitcoin-cash,1.193036,2.000959


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [191]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [192]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
for k in k_values:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=k, random_state=42)
    
    # 2. Fit the model to the data using `df_market_scaled`
    model.fit(df_market_scaled)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia.append(model.inertia_)

In [193]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k_values, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)


In [194]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.

# Plot Elbow Curve with renamed columns
df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xlabel="Number of Clusters (k)",
    ylabel="Inertia",
)


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 
4 would be the best value for 'k'

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [195]:
# Initialize the K-Means model using the best value for k
# Initialize the K-Means model using the best value for k
from sklearn.cluster import KMeans

optimal_k = 4  # Replace this with the value you determined from the Elbow Curve
kmeans_model = KMeans(n_clusters=optimal_k, random_state=42)

In [196]:
# Fit the K-Means model using the scaled DataFrame
kmeans_model.fit(df_market_scaled)


KMeans(n_clusters=4, random_state=42)

In [197]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
cluster_labels = kmeans_model.predict(df_market_scaled)

# Print the resulting array of cluster values.
print(cluster_labels)


[0 0 1 1 2 2 2 2 0 1 1 1 1 2 0 0 1 1 2 1 1 0 1 1 1 1 0 1 0 1 0 1 3 0 0 2 1
 1 0 1 1]


In [198]:
# Create a copy of the scaled DataFrame
df_clusters = df_market_scaled.copy()

In [199]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_clusters["Cluster"] = cluster_labels


# Display the copy of the scaled DataFrame
df_clusters.head()


,price_change_percentage_24h,price_change_percentage_7d,Cluster
coin_id,,,
bitcoin,0.508529,0.493193,0
ethereum,0.185446,0.934445,0
tether,0.021774,-0.706337,1
ripple,-0.040764,-0.810928,1
bitcoin-cash,1.193036,2.000959,2


In [200]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
df_clusters.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    # Color the graph points with the labels found using K-Means
    by="Cluster",
    # Add the crypto name in the `hover_cols` parameter to identify
    # the cryptocurrency represented by each data point.
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters Based on Price Changes",
    xlabel="24-Hour Price Change (%)",
    ylabel="7-Day Price Change (%)",
    width=800,
    height=500
)

:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [201]:
from sklearn.decomposition import PCA

# Create a PCA model instance and set `n_components=2`.
pca = PCA(n_components=2)


In [202]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
pca_data = pca.fit_transform(df_market_scaled)

# View the scaled PCA data
print(pca_data[:5])  # Display first five rows


[[-0.70832473 -0.0108444 ]
 [-0.79188247  0.52962238]
 [ 0.48405906 -0.51485209]
 [ 0.60223751 -0.54458796]
 [-2.25849563  0.57128782]]


In [203]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_

# Print explained variance for each principal component
print("Explained Variance Ratio:", explained_variance)

# Calculate total explained variance
total_explained_variance = explained_variance.sum()
print("Total Explained Variance:", total_explained_variance)


Explained Variance Ratio: [0.58482974 0.41517026]
Total Explained Variance: 1.0


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 
The total explained variance is the sum of the explained variance ratios from the pca.explained_variance_ratio_ array.

In [204]:
# Create a new DataFrame with the PCA data.
# Create a new DataFrame with the PCA data.
df_pca = pd.DataFrame(
    pca_data, 
    columns=["PC1", "PC2"],  # Adjust for 2 components
    index=df_market_scaled.index
)

# Copy the crypto names from the original scaled DataFrame
df_pca["coin_id"] = df_market_scaled.index


# Set the coin_id column as index
df_pca.set_index("coin_id", inplace=True)


# Display the scaled PCA DataFrame
df_pca.head()


,PC1,PC2
coin_id,,
bitcoin,-0.708325,-0.010844
ethereum,-0.791882,0.529622
tether,0.484059,-0.514852
ripple,0.602238,-0.544588
bitcoin-cash,-2.258496,0.571288


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [205]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))


In [206]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    
    # 2. Fit the model to the data using `df_pca`
    kmeans.fit(df_pca)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans.inertia_)

In [207]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k_values, "inertia": inertia_values}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data)


# Display the first few rows
df_elbow_pca.head()

,k,inertia
0,1,82.000000
1,2,55.059288
2,3,44.076901
3,4,20.952553
4,5,15.073548


In [208]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
df_elbow_pca.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve (PCA Data)",
    xlabel="Number of Clusters (k)",
    ylabel="Inertia",
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

**Question:** What is the best value for k when using the PCA data?
  - **Answer:** The optimal k-value appears to be 4 based on the Elbow Curve

**Question:** Does it differ from the best k value found using the original data?
  - **Answer:**  Yes, it may differ. Compare it with the original data Elbow Curve. If the original dataset suggested a different k (e.g., 5 or 6), then PCA has reduced the feature dimensions, affecting clustering.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [209]:
# Initialize the K-Means model using the best value for k
best_k_pca = 4  # Updated this based on Elbow Curve
kmeans_pca = KMeans(n_clusters=best_k_pca, random_state=42)

In [210]:
# Fit the K-Means model using the PCA data
kmeans_pca.fit(df_pca)

KMeans(n_clusters=4, random_state=42)

In [211]:
# Predict the clusters
pca_clusters = kmeans_pca.predict(df_pca)

# Print the resulting array of cluster values
print(pca_clusters[:10])  # Display first 10 cluster assignments


[0 0 1 1 2 2 2 2 0 1]


In [212]:
# Create a copy of the scaled PCA DataFrame
df_pca_clusters = df_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_pca_clusters["Cluster"] = pca_clusters

# Display the copy of the scaled PCA DataFrame
df_pca_clusters.head(10)  # Show first 10 rows


,PC1,PC2,Cluster
coin_id,,,
bitcoin,-0.708325,-0.010844,0
ethereum,-0.791882,0.529622,0
tether,0.484059,-0.514852,1
ripple,0.602238,-0.544588,1
bitcoin-cash,-2.258496,0.571288,2
binancecoin,-1.569187,0.307891,2
chainlink,-1.826915,1.810797,2
cardano,-1.138818,0.993819,2
litecoin,-0.291182,0.181585,0


In [213]:
# Create a scatter plot using hvPlot
df_pca_clusters.hvplot.scatter(
    x="PC1", 
    y="PC2", 
    by="Cluster",  # Color points based on the K-Means cluster labels
    hover_cols=["coin_id"],  # Add cryptocurrency name to hover tool
    title="Cryptocurrency Clusters (PCA Data)",
    xlabel="Principal Component 1 (PC1)",
    ylabel="Principal Component 2 (PC2)",
    width=800,
    height=500
)

:NdOverlay   [Cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [214]:
# Composite plot to contrast the Elbow curves


# Recompute inertia for original data
k_values = list(range(1, 11))
inertia_values = []

for k in k_values:
    model = KMeans(n_clusters=k, random_state=1)
    model.fit(df_market_scaled)  # Ensure this is the correct DataFrame
    inertia_values.append(model.inertia_)

df_elbow = pd.DataFrame({"k": k_values, "inertia": inertia_values})

print(df_elbow)


    k    inertia
0   1  82.000000
1   2  50.727034
2   3  30.299825
3   4  22.365780
4   5  16.885359
5   6  13.635244
6   7  11.322089
7   8   7.413626
8   9   6.354520
9  10   5.498619


In [215]:

# Overlay both elbow curves
elbow_comparison = df_elbow.hvplot.line(
    x="k", y="inertia", label="Original Data", line_width=2
) * df_elbow_pca.hvplot.line(
    x="k", y="inertia", label="PCA Data", line_width=2, color="orange"
)

# Display the comparison
elbow_comparison.opts(
    title="Elbow Curve Comparison (Original vs. PCA Data)",
    xlabel="Number of Clusters (k)",
    ylabel="Inertia",
)


:Overlay
   .Curve.Original_Data :Curve   [k]   (inertia)
   .Curve.PCA_Data      :Curve   [k]   (inertia)

In [216]:
# Create a copy of the scaled DataFrame
df_market_scaled_clusters = df_market_scaled.copy()

# Add the predicted clusters as a new column
df_market_scaled_clusters["Cluster"] = model.labels_

# Display the DataFrame to confirm
print(df_market_scaled_clusters.head())



              price_change_percentage_24h  price_change_percentage_7d  Cluster
coin_id                                                                       
bitcoin                          0.508529                    0.493193        4
ethereum                         0.185446                    0.934445        9
tether                           0.021774                   -0.706337        3
ripple                          -0.040764                   -0.810928        3
bitcoin-cash                     1.193036                    2.000959        7


In [217]:
print(df_market_scaled.shape)


(41, 2)


In [218]:
# Ensure PCA is applied and stored in a DataFrame
pca = PCA(n_components=2)
pca_data = pca.fit_transform(df_market_scaled)  # Ensure df_market_scaled exists

# Convert the PCA-transformed data into a DataFrame
df_market_pca = pd.DataFrame(
    pca_data, 
    columns=["PC1", "PC2"], 
    index=df_market_scaled.index  # Keep the original index
)

# Display the DataFrame
print(df_market_pca.head())


                   PC1       PC2
coin_id                         
bitcoin      -0.708325 -0.010844
ethereum     -0.791882  0.529622
tether        0.484059 -0.514852
ripple        0.602238 -0.544588
bitcoin-cash -2.258496  0.571288


In [219]:
# Create a copy of the PCA-transformed DataFrame
df_market_pca_clusters = df_market_pca.copy()

# Assign the same cluster labels used earlier
df_market_pca_clusters["Cluster"] = model.labels_  # Use the same trained model

# Display the DataFrame to confirm
print(df_market_pca_clusters.head())


                   PC1       PC2  Cluster
coin_id                                  
bitcoin      -0.708325 -0.010844        4
ethereum     -0.791882  0.529622        9
tether        0.484059 -0.514852        3
ripple        0.602238 -0.544588        3
bitcoin-cash -2.258496  0.571288        7


In [220]:
# Composite plot to contrast the clusters
# Scatter plot for clusters using original scaled data
original_clusters_plot = df_market_scaled_clusters.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters (Original Data)"
)

# Scatter plot for clusters using PCA-transformed data
pca_clusters_plot = df_market_pca_clusters.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Cluster",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters (PCA Data)"
)

# Combine both plots for comparison
comparison_plot = original_clusters_plot + pca_clusters_plot

# Display the comparison
comparison_plot.opts(shared_axes=False)


:Layout
   .NdOverlay.I  :NdOverlay   [Cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Cluster]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 
  Reducing the dataset to fewer features using PCA:

Improves computational efficiency: PCA reduces dimensionality, making clustering faster.
Enhances cluster separation: It removes noise and redundant information, potentially leading to clearer clusters.
May change cluster structure: Some details are lost when reducing features, possibly altering cluster assignments compared to the original data.
Your visualization shows this effect:

The Original Data Clusters (left) have more spread and complexity.
The PCA Data Clusters (right) show a more compact and structured distribution.